## Commuting Distance Visualisation
### Wrangling & Data Checking


In [0]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline
from IPython.display import display
import re

In [0]:
# load dataset
df_vic = pd.read_excel('Vic regional data.xlsx')
df_occ = pd.read_excel('commuting distance by personal characteristics.xls', sheet_name='Table 3')
df_ind = pd.read_excel('commuting distance by personal characteristics.xls', sheet_name='Table 4')
df_inc = pd.read_excel('commuting distance by personal characteristics.xls', sheet_name='Table 5')
df_met = pd.read_excel('commuting distance by personal characteristics.xls', sheet_name='Table 8')
df_sub = pd.read_excel('commuting distance from usual residence.xls', sheet_name='Table 8')

In [0]:
print(df_vic.shape)
print(df_occ.shape)
print(df_ind.shape)
print(df_inc.shape)
print(df_met.shape)
print(df_sub.shape)

(2932, 288)
(1050, 9)
(2429, 9)
(1969, 9)
(1855, 9)
(2305, 7)


### 1. Wrangling Victorian regional dataset

In [0]:
df = df_vic.copy()
# size
print(df.shape)
# inspect dataset
df.head()

(2932, 288)


,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,...,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,Unnamed: 281,Unnamed: 282,Unnamed: 283,Unnamed: 284,Unnamed: 285,Unnamed: 286,Unnamed: 287
0,NaN,Data 2016,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,POPULATION,Population,Persons aged 0-14,Persons aged 15-24,Persons aged 12-24,Persons aged 65+,Persons aged 70+,Persons aged 85+,BIRTHPLACE,Persons born in Afghanistan,Persons born in Australia,Persons born in Bangladesh,Persons born in Bosnia and Herzegovina,Persons born in Cambodia,Persons born in Chile,Persons born in China,Persons born in Croatia,Persons born in Cyprus,Persons born in Egypt,Persons born in England,Persons born in Ethiopia,Persons born in Fiji,Persons born in France,Persons born in Germany,Persons born in Greece,Persons born in Hong Kong,Persons born in India,Persons born in Indonesia,Persons born in Iran,Persons born in Iraq,Persons born in Ireland,Persons born in Italy,Persons born in Japan,"Persons born in Korea, South",Persons born in Lebanon,Persons born in Malaysia,Persons born in Mauritius,Persons born in Myanmar,...,Mitchell,Moira,Monash,Moonee Valley,Moorabool,Moreland,Mornington Peninsula,Mount Alexander,Moyne,Murrindindi,Nillumbik,Northern Grampians,Port Phillip,Pyrenees,Queenscliffe,South Gippsland,Southern Grampians,Stonnington,Strathbogie,Surf Coast,Swan Hill,Towong,Wangaratta,Warrnambool,Wellington,West Wimmera,Whitehorse,Whittlesea,Wodonga,Wyndham,Yarra,Yarra Ranges,Yarriambiak,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1.0,Abbeyard,NaN,0,0,0,0,0,0,0,NaN,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Alpine
4,2.0,Abbotsford (Vic.),NaN,8184,644,971,1062.2,705,453,101,NaN,0,4510,11,0,6,19,224,10,0,10,308,5,5,34,38,96,53,142,36,29,0,50,46,34,28,0,147,9,6,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Ararat


#### 1.1. Drop irrelevant rows and columns

In [0]:
# extract suburb column
vic_suburbs = df.iloc[3:,1]
# extract relevant columns
df2 = df.iloc[:,118:131].copy()
df2.drop([124,125,126,127,128],axis=1,inplace=True)
# combine suburb column and relevant columns
df3 = pd.concat([vic_suburbs,df2],axis=1)
# drop irrelevant rows
df3.drop([0,1],axis=0,inplace=True)
df3.head()

,2,119,120,121,122,123,129,130,131
2,NaN,Couple families with no children,Couple familles with children,One parent families,Other families,Lone persons,Private dwellings rented by their occupants,Private dwellings owned or being purchased by ...,Private dwellings: rented from govt. or charit...
3,Abbeyard,0,0,0,0,0,0,0,0
4,Abbotsford (Vic.),1160,492,159,75,1164,2082,1413,133
5,Aberfeldie,290,601,135,16,253,286,989,27
6,Aberfeldy,0,0,0,0,0,0,0,0


#### 1.2. Cluster similar columns

In [0]:
# cluster & rename columns
df3['Lives with families'] = df3.loc[:,119] + df3.loc[:,120] + df3.loc[:,121] \
                            + df3.loc[:,122] 
df3['Lives alone'] = df3.loc[:,123]
df3['Rents property'] = df3.loc[:,129] + df3.loc[:,131]
df3['Owns property'] = df3.loc[:,130]
df4 = df3.copy()
df4.drop([119,120,121,122,123,129,130,131],axis=1,inplace=True)
df4.drop(2,axis=0,inplace=True)
df4.columns = ['Suburbs', 'Lives with families', 'Lives alone', \
              'Rents property', 'Owns property']

# reset index
df4.reset_index(drop=True, inplace=True)
print(df4.shape)
df4.head()

(2929, 5)


,Suburbs,Lives with families,Lives alone,Rents property,Owns property
0,Abbeyard,0,0,0,0
1,Abbotsford (Vic.),1886,1164,2215,1413
2,Aberfeldie,1042,253,313,989
3,Aberfeldy,0,0,0,0
4,Acheron,41,14,4,47


#### 1.4 Export Vic regional data

In [0]:
# export vic regional data
df4.to_csv("Vic property & family.csv", encoding='utf-8')

### 2. Wrangling Commuting distance by personal characteristics

In [0]:
# inspect dataset
df_occ.head(6)

,Australian Bureau of Statistics,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8
0,2071.0.55.001 Census of Population and Housing...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Released at 11.30am (Canberra time) 22 May 2018,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Table 3. Commuting distance from Place of Usua...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Australian Statistical Geography Standard (ASG...,Australian Statistical Geography Standard (ASG...,Australian Statistical Geography Standard (ASG...,Occupation (OCCP),Number of employed people living in region (no.),Average commuting distance \n(kilometres),Median commuting distance \n(kilometres),Interquartile range (kilometres),Standard deviation (kilometres)
5,AUST,036,Australia,Managers,1336720,16.7,10.16,18.03,22.95


#### 2.1. Remove irrelevant rows and columns

In [0]:
# for occupation sheet
# drop rows
df_occ2 = df_occ.drop([0,1,2,3], axis=0)
# drop columns
df_occ3 = df_occ2.drop(['            Australian Bureau of Statistics', 'Unnamed: 1',
          'Unnamed: 4', 'Unnamed: 6', 'Unnamed: 7', 'Unnamed: 8'], axis=1)
# rename columns
df_occ3.columns = ['Region', 'Occupation', 'Average distance (km)']
# drop previously column row
df_occ4 = df_occ3.drop(4, axis=0)
display(df_occ4.head())

# for industry sheet
# drop rows
df_ind2 = df_ind.drop([0,1,2,3], axis=0)
# drop columns
df_ind3 = df_ind2.drop(['            Australian Bureau of Statistics', 'Unnamed: 1',
          'Unnamed: 4', 'Unnamed: 6', 'Unnamed: 7', 'Unnamed: 8'], axis=1)
# rename columns
df_ind3.columns = ['Region', 'Industry', 'Average distance (km)']
# drop previously column row
df_ind4 = df_ind3.drop(4, axis=0)
display(df_ind4.head())

# for income sheet
# drop rows
df_inc2 = df_inc.drop([0,1,2,3], axis=0)
# drop columns
df_inc3 = df_inc2.drop(['            Australian Bureau of Statistics', 'Unnamed: 1',
          'Unnamed: 4', 'Unnamed: 6', 'Unnamed: 7', 'Unnamed: 8'], axis=1)
# rename columns
df_inc3.columns = ['Region', 'Income (AUD)', 'Average distance (km)']
# drop previously column row
df_inc4 = df_inc3.drop(4, axis=0)
display(df_inc4.head())

# for method sheet
# drop rows
df_met2 = df_met.drop([0,1,2,3], axis=0)
# drop columns
df_met3 = df_met2.drop(['            Australian Bureau of Statistics', 'Unnamed: 1',
          'Unnamed: 6', 'Unnamed: 7', 'Unnamed: 8'], axis=1)
# rename columns
df_met3.columns = ['Region', 'Travel method', 'Number of people', 'Average distance (km)']
# drop previously column row
df_met4 = df_met3.drop(4, axis=0)
display(df_met4.head())

# for method sheet - all Vic suburbs
# drop rows
df_met_vic2 = df_met.drop([0,1,2,3], axis=0)
# drop columns
df_met_vic3 = df_met_vic2.drop(['            Australian Bureau of Statistics',
          'Unnamed: 6', 'Unnamed: 7', 'Unnamed: 8'], axis=1)
# rename columns
df_met_vic3.columns = ['Area code', 'Region', 'Travel method', 'Number of people', 'Average distance (km)']
# drop previously column row
df_met_vic4 = df_met_vic3.drop(4, axis=0)
display(df_met_vic4.head())

,Region,Occupation,Average distance (km)
5,Australia,Managers,16.7
6,Australia,Professionals,15.63
7,Australia,Technicians and Trades Workers,18.22
8,Australia,Community and Personal Service Workers,14.22
9,Australia,Clerical and Administrative Workers,15.52


,Region,Industry,Average distance (km)
5,Australia,"Agriculture, Forestry and Fishing",21.09
6,Australia,Mining,40.27
7,Australia,Manufacturing,18.37
8,Australia,"Electricity, Gas, Water and Waste Services",21.34
9,Australia,Construction,18.49


,Region,Income (AUD),Average distance (km)
5,Australia,Negative income,14.03
6,Australia,Nil income,12.96
7,Australia,"$1-$149 ($1-$7,799)",9.63
8,Australia,"$150-$299 ($7,800-$15,599)",11.63
9,Australia,"$300-$399 ($15,600-$20,799)",12.27


,Region,Travel method,Number of people,Average distance (km)
5,Australia,Train,755359,22.49
6,Australia,Bus,342752,13.1
7,Australia,Ferry,16509,15.55
8,Australia,Tram,64510,7
9,Australia,Taxi,21896,9.73


,Area code,Region,Travel method,Number of people,Average distance (km)
5,036,Australia,Train,755359,22.49
6,036,Australia,Bus,342752,13.1
7,036,Australia,Ferry,16509,15.55
8,036,Australia,Tram,64510,7
9,036,Australia,Taxi,21896,9.73


#### 2.2. Select Greater Melbourne and Rest of Victoria datasets


In [0]:
# Select Greater Melbourne and Rest of Victoria datasets
# for occupation 
df_occ5 = df_occ4.loc[df_occ4.Region.isin(['Greater Melbourne','Rest of Vic.'])]
# reset_index
df_occ5.reset_index(drop=True, inplace=True)

# for industry
df_ind5 = df_ind4.loc[df_ind4.Region.isin(['Greater Melbourne','Rest of Vic.'])]
df_ind6 = df_ind5.loc[~df_ind5.Industry.isin(['Inadequately described/Not stated'])]
# reset_index
df_ind6.reset_index(drop=True, inplace=True)

# for income
df_inc5 = df_inc4.loc[df_inc4.Region.isin(['Greater Melbourne','Rest of Vic.'])]
df_inc6 = df_inc5.loc[~df_inc5['Income (AUD)'].isin(['Negative income','Not stated'])].copy()
# conver income unit to annual income in AUD
def income_convert(value):
    if value == "Nil income":
        value = '0'
        return value
    elif value == "Total":
        return value
    else:
        val = re.findall('\(.*?\)',value)
        value = val[0]
        value = value.replace('(','')
        value = value.replace(')','')
        value = value.replace('$','')
        return value
df_inc6['Income (AUD)'] = df_inc6['Income (AUD)'].apply(lambda x: income_convert(x))
# reset_index
df_inc6.reset_index(drop=True, inplace=True)

# for method
df_met5 = df_met4.loc[df_met4.Region.isin(['Greater Melbourne','Rest of Vic.'])]
df_met6 = df_met5.loc[~df_met5['Travel method'].\
                      isin(['Worked at home (e)','Did not go to work (e)',
                            'Not stated','Total'])]
# reset_index
df_met6.reset_index(drop=True, inplace=True)
df_met6

# for method - all Vic suburbs
def vic_sub(value):
    value = str(value)
    if value[0] == '2' and len(value) == 3:
        return value
    else:
        return 'na'

df_met_vic5 = df_met_vic4.copy()
df_met_vic5['Area code'] = df_met_vic4['Area code'].apply(lambda x: vic_sub(x))
df_met_vic6 = df_met_vic5.loc[df_met_vic5['Area code'] != 'na']
df_met_vic7 = df_met_vic6.loc[~df_met_vic6['Travel method'].\
                      isin(['Worked at home (e)','Did not go to work (e)',
                            'Not stated','Total'])]
# drop area code column
df_met_vic8 = df_met_vic7.drop('Area code', axis=1)
# reset_index
df_met_vic8.reset_index(drop=True, inplace=True)

#### 2.3. Export Commuting distance by personal characteristics

In [0]:
# export occupation
df_occ5.to_csv("Commuting distance by occupation.csv", encoding='utf-8')

# export industry
df_ind6.to_csv("Commuting distance by industry.csv", encoding='utf-8')

# export income
df_inc6.to_csv("Commuting distance by income.csv", encoding='utf-8')

# export method
df_met6.to_csv("Commuting distance by travel method.csv", encoding='utf-8')

# export method - all Vic suburbs
df_met_vic8.to_csv("Commuting distance by travel method - all Vic suburbs.csv", encoding='utf-8')

In [0]:
print(df_occ5.shape)
print(df_ind6.shape)
print(df_inc6.shape)
print(df_met6.shape)
print(df_met_vic8.shape)

(18, 3)
(40, 3)
(30, 3)
(24, 4)
(204, 4)


### 3. Wrangling Commuting distance from usual residence

In [0]:
# inspect dataset
df_sub.head()

,Australian Bureau of Statistics,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6
0,2071.0.55.001 Census of Population and Housing...,NaN,NaN,NaN,NaN,NaN,NaN
1,Released at 11.30am (Canberra time) 22 May 2018,NaN,NaN,NaN,NaN,NaN,NaN
2,Table 8. Commuting distance from Place of Usua...,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Statistical Area Level 2 code,Statistical Area Level 2 label,Number of employed people living in region (no.),Average commuting distance (kms),Median commuting distance (kms),Interquartile range (kms),Standard deviation (kms)


#### 3.1. Remove irrelevant rows and columns

In [0]:
# drop rows
df_sub2 = df_sub.drop([0,1,2,3], axis=0)
# drop columns
df_sub3 = df_sub2.drop(['Unnamed: 2', 'Unnamed: 4', 'Unnamed: 5', 'Unnamed: 6'], axis=1)
# rename columns
df_sub3.columns = ['Area code', 'Area', 'Average distance (km)']
# drop previously column row
df_sub4 = df_sub3.drop(4, axis=0)
print(df_sub4.shape)
display(df_sub4.head())

(2300, 3)


,Area code,Area,Average distance (km)
5,101021007,Braidwood,37.95
6,101021008,Karabar,11.55
7,101021009,Queanbeyan,10.66
8,101021010,Queanbeyan - East,12.25
9,101021011,Queanbeyan Region,28.18


#### 3.2. Select Victorian suburbs 


In [0]:
# select Victorian suburbs dataset
# area code for Victorian suburbs starts with '2'
def select_vic(value):
    value = str(value)
    if len(value) != 9:
        return 'na'
    elif value[0] != '2':
        return 'na'
    else:
        return (int(value))

df_sub5 = df_sub4.copy()
df_sub5['Area code'] = df_sub5['Area code'].apply(lambda x: select_vic(x))
df_sub6 = df_sub5.loc[df_sub5['Area code'] != 'na']

# remove area code column
df_sub7 = df_sub6.drop('Area code', axis=1).copy()

# reset index
df_sub7.reset_index(drop=True, inplace=True)

#### 3.3. Export Commuting distance from usual residence dataset

In [0]:
# export dataset
df_sub7.to_csv("Commuting distance from residence.csv", encoding='utf-8')

In [0]:
df_sub7.shape

(462, 2)